# Prelims

In [ ]:
# Install the Google Earth Engine Python API and geemap, if necessary
!pip install earthengine-api
!pip install geemap

In [ ]:
import geemap
from calendar import monthrange

gabamVis = {'palette': ['f00c0c'], 'min':1, 'max':1} # Generic binary mask for visualisation

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='spherical-berm-323321') # <-- Edit project linked to your individual GEE account

In [ ]:
# For boreal and tundra Northern hemisphere individual ecoregions
# Load the ecoregions
ecoRegions = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017')

# Apply filters
biome_filter = ee.Filter.inList('BIOME_NUM', [6])
realm_filter = ee.Filter.inList('REALM', ['Nearctic', 'Palearctic'])
combined_filter = ee.Filter.And(biome_filter, realm_filter)
selected_regions = ecoRegions.filter(combined_filter)
region_list = selected_regions.toList(selected_regions.size())

# Area analysis for RESOLVE ecoregions

In [ ]:
# Use this and next cell to find ecoregions where BA /= 0, or to view on map
# import FireCCI data (global) - from 2001 to 2020
dataset = (ee.ImageCollection("ESA/CCI/FireCCI/5_1")
    .filterDate('2001-01-01', '2020-12-31'))

# created burned area and confidence level layers
julianDay = dataset.select('BurnDate')
confLevel = dataset.select('ConfidenceLevel')
landCover = dataset.select('LandCover')

maxJD = julianDay.max()
maxCL = confLevel.max()

CLmask = maxCL.gte(ee.Number(80))
maxJDmasked = maxJD.mask(CLmask)

#final = ee.FeatureCollection('users/andyc97/model_shapefiles/south_siberia_final')
#final = ee.FeatureCollection('users/andyc97/model_shapefiles/final_north')
reprojected = maxJDmasked.reproject(crs='EPSG:6931', scale=4000)
binary = reprojected.multiply(0).add(1).clip(selected_regions)

In [ ]:
for i in range(region_list.size().getInfo()):
    feature = ee.Feature(region_list.get(i))
    eco_name = feature.get('ECO_NAME').getInfo()

    # Generate a short name for the region
    words = eco_name.split()
    if len(words) >= 2:
        short_name = (words[0][:4] + words[1][:3]).lower()
    else:
        short_name = words[0][:7].lower()

    print(f"Processing region: {eco_name} -> {short_name}")

    pa = ee.Image.pixelArea()

    # Get area
    img_area = pa.updateMask(binary).reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry=feature.geometry(),
      scale=4000, bestEffort=False,
      crs='EPSG:6931',
      maxPixels=1e12).get('area')
    pixel_count = img_area.getInfo() / 1e10 # Area in Mha
    print("Region =", short_name)
    print("Pixel count =", pixel_count)

In [ ]:
# Use this and next cell to run over ecoregions where BA /= 0
fire_dataset = ee.ImageCollection("ESA/CCI/FireCCI/5_1") \
    .filterDate('2020-01-01', '2020-12-31') \
    .select(['BurnDate', 'ConfidenceLevel'])

# Confidence mask
conf_masked = fire_dataset.map(
    lambda img: img.updateMask(img.select('ConfidenceLevel').gte(80)))

In [ ]:
# Loop over regions
for i in [2,4,5,6,7,8,9,10,14,15,17,18,19,20,22,23,24,25,28,31,43,47,48,49,51,54,55,57]: # <-- edit as necessary
    feature = ee.Feature(region_list.get(i))
    eco_name = feature.get('ECO_NAME').getInfo()

    # Generate mnemonic name
    words = eco_name.split()
    if len(words) >= 2:
        short_name = (words[0][:4] + words[1][:3]).lower()
    else:
        short_name = words[0][:7].lower()

    print(f"Processing region: {eco_name} -> {short_name}")

    # Loop through months
    for month in range(1, 13):
        start_date = f'2020-{month:02d}-01'
        end_day = monthrange(2001, month)[1]
        end_date = f'2020-{month:02d}-{end_day}'

        # Filter the dataset to the month
        month_images = conf_masked.filterDate(start_date, end_date)
        maxJD = month_images.select('BurnDate').max()

        # Make binary mask
        binary = maxJD.multiply(0).add(1).reproject(crs='EPSG:6931', scale=4000)

        # Get pixel area
        pixel_area = ee.Image.pixelArea().updateMask(binary)

        # Sum burned area in region
        burned = pixel_area.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=feature.geometry(),
            scale=4000,
            crs='EPSG:6931',
            maxPixels=1e12
        ).get('area')

        burned_mha = burned.getInfo() / 1e10 if burned else 0  # Convert m² to Mha
        print(f"  {start_date}: {burned_mha} Mha")

In [ ]:
# import MCD64A1 dataset (global) - from 2001 to present
dataset2 = (ee.ImageCollection("MODIS/061/MCD64A1")
    .filterDate('2021-01-01', '2023-12-31'))

# created burned area and uncertainty layers
burnDate = dataset2.select('BurnDate')
uncertainty = dataset2.select('Uncertainty')

maxBD = burnDate.max()
maxUC = uncertainty.max()

#final = ee.FeatureCollection('users/andyc97/model_shapefiles/south_siberia_final')
final = ee.FeatureCollection('users/andyc97/model_shapefiles/final_north')
reprojected = maxBD.reproject(crs='EPSG:6931', scale=4000)
binary_mcd = reprojected.multiply(0).add(1).clip(selected_regions)

In [ ]:
for i in range(region_list.size().getInfo()):
    feature = ee.Feature(region_list.get(i))
    eco_name = feature.get('ECO_NAME').getInfo()

    # Generate a short name for the region
    words = eco_name.split()
    if len(words) >= 2:
        short_name = (words[0][:4] + words[1][:3]).lower()
    else:
        short_name = words[0][:7].lower()

    print(f"Processing region: {eco_name} -> {short_name}")

    pa = ee.Image.pixelArea()

    # Get area
    img_area = pa.updateMask(binary).reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry=feature.geometry(),
      scale=4000, bestEffort=False,
      crs='EPSG:6931',
      maxPixels=1e12).get('area')
    pixel_count = img_area.getInfo() / 1e10 # Area in Mha
    print("Region =", short_name)
    print("Pixel count =", pixel_count)

In [ ]:
Map = geemap.Map()
Map.add_basemap("Esri.WorldImagery")
Map.addLayer(binary, gabamVis, 'FireCCI')
Map.addLayer(binary_mcd, gabamVis, 'MCD')
Map